In [3]:
from time import time
import numpy as np
from tst.domutil.util import *
from tst.domutil.test import *
from copy import copy
from scipy.sparse import *
from scipy.spatial.distance import *


def normalise(hc1, hc2, hcboth):
    log1 =  np.log10( hc1 + 1) 
    log2 =  np.log10( hc1 + 1) 
    log3 =  np.log10( hcboth + 1) 
    norm = (log1 + log2) / 2 - log3 
    return norm

reset_database_connection()

In [13]:
# import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

In [17]:
import sys
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *

In [2]:
%%time
# print "finished"
hmms = HMMprofile.objects.defer('text').all().prefetch_related('hits')
hmms = hmms.annotate(hcount = Count('hits')).order_by('id')

hmmids = list(hmms.values_list('id',flat = True))
hcounts = list(hmms.values_list('hcount',flat = True))
sfs = list(hmms.values_list('cath_node__parent',flat = True))

sDB = seqDB.objects.get(name = 'CATH')
sDB.sequence_set.count()
seq = sDB.sequence_set.first()

seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set')

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
seq.hmmprofile_set.count()


CPU times: user 152 ms, sys: 4 ms, total: 156 ms
Wall time: 45.7 s


In [ ]:
%%time
X0 = hmms2hit_ids(hmms)

In [8]:
%%time

# t0 = time()
# c = counter(range(len(X)**2/2),per = 1E3)
# from multiprocessing import *

hmms_test = HMMprofile.objects.defer('text').all().prefetch_related('hits')[:200]
# hmms_test = HMMprofile.objects.defer('text').all().prefetch_related('hits')[:500]
X = hmms2hit_ids( hmms_test )

CPU times: user 324 ms, sys: 12 ms, total: 336 ms
Wall time: 442 ms


In [ ]:
from tst.domutil.util import *
# its = D.iteritems()
its = using_tocoo_izip(D)

xs = []
ys = []
zs = []
sfbools = [];

c = counter(range(6563388),per = 100000)

for xi,yi,inter_ct in its:
    c.count()
#     xct = 
    x = hcounts[xi]
    y = hcounts[yi]

    xs.append( x )
    ys.append( y )
    zs.append(inter_ct)
    sfbools.append(sfs[xi]==sfs[yi])
#     normz = log10()
# print next(its)
# print next(its)

# ys 

In [31]:
    from scipy.sparse import *
    D = coo_matrix([])

In [4]:
print seqset_all.count()

80407


# Counting HMM-HMM intermediate sequence
## (sequence wise, Parallel)

In [3]:
#### Defining Parallel Wrappers ####

import itertools as it
from collections import Counter

def pair_gen(seqset_all):
    c = counter(range(seqset_all.count()))
    batches = batch_qs( seqset_all, 500)
    for seqset in batches:
        
#             hmmids = seq.values_list("hmmprofile",flat =True)
        for seq in seqset:
            c.count()
            hmmids = seq.hmmprofile_set.values_list("id",flat = True)
#             pairs = it.combinations(sorted(hmmids),2)
            pairs = it.combinations( hmmids ,2)
            for p in pairs:
                yield p

def batch_worker(seqset):
    count = Counter()
    for seq in seqset:
        c0.count()
        hmmids = seq.hmmprofile_set.values_list("id",flat = True)
    #             pairs = it.combinations(sorted(hmmids),2)
        pairs = it.combinations( hmmids ,2)
        count.update(pairs)
    q.put(count)
#     c1.count()
    return 


def seq_counter(seq):
    c0.count()
    hmmids = seq.hmmprofile_set.values_list("id",flat = True)
#             pairs = it.combinations(sorted(hmmids),2)
    pairs = it.combinations( hmmids ,2)
    count = Counter(pairs)
    q.put(count)
    
    
def listener( OUTPUT ):
#     OUTPUT should be a dok matrix !!!!
#     D_curr = dok_matrix( OUTPUT )
    counts = Counter([])    
    i = 0

    while 1:
        print i
        i += 1
        obj = q.get()
        if obj:
            c1.count()
            counts.update(obj)            
        else:
            ################################################################
            ###### Put counts into a sparse matrix #########################
            ################################################################     
#             OUTPUT = OUTPUT.todok()
            OUTPUT.update(counts)
#             q.put( OUTPUT.todok() ) ####### DOK matrix breaks if put on a Queue() for unknown reason
            q.put( OUTPUT.tocoo() )
#             q.put(counts)
            break
    return
    
def listener_mat( OUTPUT ):
#     from scipy.sparse import *
    D_curr = OUTPUT
    i = 0
    while 1:
        print i
        i += 1
        obj = q.get()
        if obj:
            c1.count()
            for k,v in obj.iteritems():
                D_curr[k] += v
        else:
            q.put( D_curr )
            return



In [4]:
%%time

##### Actually firing subprocesses #####

from scipy.sparse import *
import multiprocessing as mp
from multiprocessing.managers import BaseManager, SyncManager
class MyManager(SyncManager): pass
MyManager.register('counter',counter)


# c = counter(range(sDB.sequence_set.count()))

pcount = 5
# INPUT = seqset_all[:1000]
INPUT = seqset_all
ct1 = INPUT.count()
bsize = 600 
INPUTs = batch_qs( INPUT, bsize )

l = max(hmmids)


####Deprecated# OUTPUT = Counter()
OUTPUT = dok_matrix( (l + 1, l + 1), dtype = 'int')
# local_listener = listener_mat      
local_listener = listener      

local_worker = batch_worker
# local_worker = seq_counter

if __name__=='__main__':
    
    #     m = mp.Manager()
        m = MyManager()
        m.start()
    #     raise Exception
    #     manager.start()
        c0 = m.counter( range(ct1),INF=0, ifprint = 1)
        c1 = m.counter( [],INF=1, ifprint = 1, prefix = 'Listener:',
                      step = bsize, per = bsize, )

        q = m.Queue();          

        if 1:
            pool = mp.Pool( pcount )
            watcher  = mp.Process(target = local_listener, args = [OUTPUT])
            watcher.start()
#             watcher  = pool.apply_async(local_listener, args = [OUTPUT])
            jobs = []

            for INPUT_curr in INPUTs:
                job  = pool.apply_async(local_worker, args = (INPUT_curr, ))
                jobs.append(job)
            for job in jobs:
                job.get()

            q.put(None)
#             OUTPUT = watcher.get()
            watcher.join())
            OUTPUT = q.get() 
        pool.close()
        pool.join()
        

0
0 of 80407
100 of 80407
200 of 80407
300 of 80407
400 of 80407
500 of 80407
800 of 80407
600 of 80407
700 of 80407
 700 of 80407
900 of 80407
1000 of 80407
 1000 of 80407
1100 of 80407
1200 of 80407
Listener:0 of -1
1
1300 of 80407
1500 of 80407
1400 of 80407
Listener:600 of -1
2
1600 of 80407
1700 of 80407
1800 of 80407
 1800 of 80407
2100 of 80407
1900 of 80407
2000 of 80407
2200 of 80407
2300 of 80407
2400 of 80407
 2400 of 80407
 2400 of 80407
2500 of 80407
 2500 of 80407
2600 of 80407
2700 of 80407
 2700 of 80407
2900 of 80407
2800 of 80407
3000 of 80407
Listener:1200 of -1
3100 of 80407
3
3200 of 80407
 3400 of 80407
3300 of 80407
Listener:1800 of -1
3400 of 80407
3500 of 80407
4
3600 of 80407
3700 of 80407
 3600 of 80407
Listener:2400 of -1
3800 of 80407
5
3900 of 80407
 3900 of 80407
4000 of 80407
 4000 of 80407
4100 of 80407
4200 of 80407
4400 of 80407
4300 of 80407
4500 of 80407
Listener:3000 of -1
4600 of 80407
4700 of 80407
4800 of 80407
4900 of 80407
6
 4900 of 80407
500

32100 of 80407
50
32200 of 80407
32300 of 80407
32400 of 80407
 32400 of 80407
32500 of 80407
Listener:30000 of -1
32600 of 80407
51
32700 of 80407
 32700 of 80407
 33000 of 80407
32800 of 80407
 32800 of 80407
 32800 of 80407
32900 of 80407
33000 of 80407
33100 of 80407
Listener:30600 of -1
33200 of 80407
 33200 of 80407
33300 of 80407
 33300 of 80407
33400 of 80407
 33400 of 80407
33500 of 80407
33600 of 80407
 33600 of 80407
33700 of 80407
33800 of 80407
52
33900 of 80407
Listener:31200 of -1
34000 of 80407
 34100 of 80407
 34100 of 80407
34100 of 80407
34200 of 80407
 34200 of 80407
34300 of 80407
53
 34300 of 80407
34400 of 80407
34500 of 80407
Listener:31800 of -1
 34600 of 80407
34600 of 80407
34700 of 80407
 34700 of 80407
34800 of 80407
54
34900 of 80407
 34900 of 80407
 34900 of 80407
 34900 of 80407
35000 of 80407
Listener:32400 of -1
35100 of 80407
35200 of 80407
55
Listener:33000 of -1
35300 of 80407
 35300 of 80407
35400 of 80407
35500 of 80407
35600 of 80407
35700 of 804

62900 of 80407
63000 of 80407
 63000 of 80407
63100 of 80407
 63100 of 80407
 63100 of 80407
63200 of 80407
Listener:60600 of -1
63300 of 80407
 63300 of 80407
63400 of 80407
63500 of 80407
63600 of 80407
102
63700 of 80407
63900 of 80407
 63700 of 80407
 63700 of 80407
 63700 of 80407
Listener:61200 of -1
63800 of 80407
64000 of 80407
103
64100 of 80407
 64100 of 80407
Listener:61800 of -1
104
64200 of 80407
 64200 of 80407
64300 of 80407
 64300 of 80407
64400 of 80407
 64400 of 80407
 64400 of 80407
64500 of 80407
Listener:62400 of -1
64600 of 80407
105
64700 of 80407
64800 of 80407
64900 of 80407
65000 of 80407
 65000 of 80407
 65000 of 80407
Listener:63000 of -1
65100 of 80407
65200 of 80407
106
65300 of 80407
65400 of 80407
65500 of 80407
 65900 of 80407
 65500 of 80407
65600 of 80407
65700 of 80407
 65700 of 80407
 65700 of 80407
65800 of 80407
Listener:63600 of -1
65900 of 80407
 65900 of 80407
66000 of 80407
66100 of 80407
66200 of 80407
107
66300 of 80407
66400 of 80407
66600 

CPU times: user 10min 23s, sys: 35.4 s, total: 10min 58s
Wall time: 14min 14s

In [6]:
test__raw( OUTPUT,hmms)
D_raw = OUTPUT

In [7]:
%%time
#### Cache the raw ISS matrix

import cPickle as pk

test__raw(D_raw,hmms)
fname = 'data/D_raw'
pk.dump(D_raw.tocoo(),open(  fname ,'wb'))

# if 'D_raw' not in locals().keys():
#     D_raw = pk.load(open(fname, 'rb'))

# test__raw(D_raw,hmms)


[OK] HMM for 3.20.20.70.102.0  against HMM for 3.40.50.80.20.0  overlaps 1, with 1 from 
0 of -1
[OK] HMM for 3.40.50.720.379.0  against HMM for 3.40.50.720.491.0  overlaps 3, with 3 from 
[OK] HMM for 3.40.50.300.138.0  against HMM for 3.40.50.2300.249.0  overlaps 9, with 9 from 
[OK] HMM for 3.40.190.10.284.0  against HMM for 3.40.190.10.186.0  overlaps 1, with 1 from 
[OK] HMM for 3.40.50.1820.101.0  against HMM for 3.90.1820.10.1.0  overlaps 1, with 1 from 
[OK] HMM for 3.20.20.70.102.0  against HMM for 3.40.50.80.20.0  overlaps 1, with 1 from 
0 of -1
[OK] HMM for 3.40.50.720.379.0  against HMM for 3.40.50.720.491.0  overlaps 3, with 3 from 
[OK] HMM for 3.40.50.300.138.0  against HMM for 3.40.50.2300.249.0  overlaps 9, with 9 from 
[OK] HMM for 3.40.190.10.284.0  against HMM for 3.40.190.10.186.0  overlaps 1, with 1 from 
[OK] HMM for 3.40.50.1820.101.0  against HMM for 3.90.1820.10.1.0  overlaps 1, with 1 from 
CPU times: user 6.26 s, sys: 224 ms, total: 6.48 s
Wall time: 6.98 s

In [6]:
# fname = 'data/D_raw'
D_raw = pk_load('D_raw')

test__raw(D_raw,hmms)


[OK] HMM for 2.60.40.10.648.0  against HMM for 2.60.40.10.369.0  overlaps 255, with 255 from 
0 of -1
[OK] HMM for 3.40.50.1820.108.0  against HMM for 3.40.50.1820.70.0  overlaps 761, with 761 from 
[OK] HMM for 3.40.50.300.171.0  against HMM for 3.40.50.2300.51.0  overlaps 10, with 10 from 
[OK] HMM for 2.60.40.10.290.0  against HMM for 2.10.70.10.35.0  overlaps 2, with 2 from 
[OK] HMM for 2.130.10.10.46.0  against HMM for 2.120.10.10.8.0  overlaps 86, with 86 from 


In [69]:
%%time
#####################################
#### Not fast enough ###########
#####################################
#####################################
# norm_count = dict()
OUTPUT = dict()
c=counter(range(6577065),per=100000)
for (x,y),v in counts.iteritems():
    c.count()
    h1 = hcounts[x-1]
    h2 = hcounts[y-1]
    h3 = v
    OUTPUT[(x,y)] = ISS_normalise(h1,h2,h3)
norm_count = OUTPUT
# for x,y,v in using_tocoo_izip(D_raw):

D_norm = copy(D_raw.todok())
D_norm.update( norm_count )

0 of 6577065
100000 of 6577065
200000 of 6577065
300000 of 6577065
400000 of 6577065
500000 of 6577065
600000 of 6577065
700000 of 6577065
800000 of 6577065
900000 of 6577065
1000000 of 6577065
1100000 of 6577065
1200000 of 6577065
CPU times: user 41 s, sys: 3.88 s, total: 44.8 s
Wall time: 40.6 s


In [8]:
%%time

###### FASTES
# norm_count = dict()
OUTPUT = dict()
# INPUT = D_raw.todok().iteritems()
INPUT = D_raw
it  = using_tocoo_izip( INPUT )

c=counter(range(6577065),per=100000)
h1s = []
h2s = []
h3s = []
for x,y,v in it:
# for (x,y),v in it:
    
    c.count()
    h1s += [hcounts[x-1]]
    h2s += [hcounts[y-1]]
    h3s += [v]

BUFlst = ISS_normalise_new(
    np.array(h1s),
    np.array(h2s),
    np.array(h3s),
)



it  = using_tocoo_izip( INPUT )
for (x,y,_),v in izip( it, BUFlst):
    k = (x,y)
    OUTPUT[k] = v

# norm_count = OUTPUT
# for x,y,v in using_tocoo_izip(D_raw):
# D_norm = copy(D_raw.todok())

l = max(hmmids) + 1
D_norm = dok_matrix( (l+1,l+1),np.float)
D_norm.update( OUTPUT )



0 of 6577065
100000 of 6577065
200000 of 6577065
300000 of 6577065
400000 of 6577065
500000 of 6577065
600000 of 6577065
700000 of 6577065
800000 of 6577065
900000 of 6577065
1000000 of 6577065
1100000 of 6577065
1200000 of 6577065
1300000 of 6577065
1400000 of 6577065
1500000 of 6577065
1600000 of 6577065
1700000 of 6577065
1800000 of 6577065
1900000 of 6577065
2000000 of 6577065
2100000 of 6577065
2200000 of 6577065
2300000 of 6577065
2400000 of 6577065
2500000 of 6577065
2600000 of 6577065
2700000 of 6577065
2800000 of 6577065
2900000 of 6577065
3000000 of 6577065
3100000 of 6577065
3200000 of 6577065
3300000 of 6577065
3400000 of 6577065
3500000 of 6577065
3600000 of 6577065
3700000 of 6577065
3800000 of 6577065
3900000 of 6577065
4000000 of 6577065
4100000 of 6577065
4200000 of 6577065
4300000 of 6577065
4400000 of 6577065
4500000 of 6577065
4600000 of 6577065
4700000 of 6577065
4800000 of 6577065
4900000 of 6577065
5000000 of 6577065
5100000 of 6577065
5200000 of 6577065
5300000 

In [9]:
%%time
import cPickle as pk

fname = 'data/D_norm'

test__norm( D_norm, hmms, ISS_normalise_new)
pk.dump(D_norm.tocoo(),open(  fname ,'wb'))

# if 'D_norm' not in locals().keys():
#     D_norm = pk.load(open(fname, 'rb'))
    
# test__norm( D_norm, hmms)

[OK] HMM for 2.60.40.10.648.0  against HMM for 2.60.40.10.369.0  overlaps:: Expected:0.983661291559, Actual:0.983661291559
0 of -1
[OK] HMM for 3.40.50.1820.108.0  against HMM for 3.40.50.1820.70.0  overlaps:: Expected:0.995279106478, Actual:0.995279106478
[OK] HMM for 3.40.50.300.171.0  against HMM for 3.40.50.2300.51.0  overlaps:: Expected:0.365878136201, Actual:0.365878136201
[OK] HMM for 2.60.40.10.290.0  against HMM for 2.10.70.10.35.0  overlaps:: Expected:0.1724558794, Actual:0.1724558794
[OK] HMM for 2.130.10.10.46.0  against HMM for 2.120.10.10.8.0  overlaps:: Expected:0.906760821212, Actual:0.906760821212
CPU times: user 18 s, sys: 1 s, total: 19 s
Wall time: 19.1 s


In [9]:
%%time
pdata_D_norm = zip(*sorted_D_norm)

CPU times: user 14.6 s, sys: 328 ms, total: 14.9 s
Wall time: 14.9 s


In [120]:
# hcounts_pad = hcounts_
hmmids_pad = hmmids[:]
hmmids_pad.insert(0, None)

hcounts_pad = hcounts[:]
hcounts_pad.insert(0,None)

sfs_pad = sfs[:]
sfs_pad.insert(0,None)

In [144]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/5000pdata'
pk.dump(pdata, open(fname, 'wb'))


In [134]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/1000pdata'
pk.dump(pdata, open(fname, 'wb'))


In [129]:
import cPickle as pk
pdata = np.vstack([xss,yss,zss])
fname = 'data/pdata_raw'
pk.dump(pdata, open(fname, 'wb'))


In [132]:
%%time

D_curr = D_norm
sorted_D_norm = sort_coo(D_curr.tocoo(),order = 1)

it = izip(
# using_tocoo_izip(D_norm),
sorted_D_norm,
sfbools,    
)

lst = sfbools
# c = counter(lst,per = 1)
D_curr = D_raw.todok()

CPU times: user 2.56 s, sys: 128 ms, total: 2.69 s
Wall time: 2.69 s


In [121]:
reset_database_connection()

In [ ]:
# view a HMM-HMM ISS_hit object
# That means we need to construct ISS_hit first

In [165]:
i = 0
c = counter([],INF = 1, per = 10 )
for (hmmid1,hmmid2,v),sfbool in it:
    c.count()   
    if sfbool:
        continue
    
    print v
    raw = D_curr[hmmid1,hmmid2]
    hmmid1 = int(hmmid1)
    hmmid2 = int(hmmid2)
    hmm1 = hmms.get(id = hmmid1 + 1)
    hmm2 = hmms.get(id = hmmid2 + 1)
#     raise
#     if hmm1.cath_node.parent == hmm2.cath_node.parent:
    cnode1 = hmm1.cath_node
    cnode2 = hmm2.cath_node
#     if cnode1.Class == cnode2.Class:
#     if cnode1.topo == cnode2.topo:
    
    if cnode1.homsf == cnode2.homsf:
        continue
    if cnode1.superfamily() in ['3.30.200.20', '1.10.10.530']:
        continue
    
#     if c.i >= 2:
    i+=1
    print hmm1
    print hmm2
    print "norm ISS is %s"%v
    print "raw ISS is %d"%raw
    print hmm1.hits.count() + hmm2.hits.count()
    
    print "http://xhits.cathdb.info/crosshits.php?sf2=%s&sf1=%s&db_source=Crosshits_v4_1_0"%(cnode1.superfamily(),cnode2.superfamily())
    if i == 10:
        break

0 of -1
0.000786053575499
HMM for 3.40.50.720.1.0 
HMM for 3.50.50.60.67.0 
norm ISS is 0.000786053575499
raw ISS is 551
1322
http://xhits.cathdb.info/crosshits.php?sf2=3.40.50.720&sf1=3.50.50.60&db_source=Crosshits_v4_1_0
0.000786053575499
HMM for 3.40.50.720.1.0 
HMM for 3.50.50.60.44.0 
norm ISS is 0.000786053575499
raw ISS is 551
1420
http://xhits.cathdb.info/crosshits.php?sf2=3.40.50.720&sf1=3.50.50.60&db_source=Crosshits_v4_1_0
10 of -1
20 of -1
30 of -1
40 of -1
50 of -1
60 of -1
70 of -1
80 of -1
90 of -1
100 of -1
110 of -1
120 of -1
130 of -1
140 of -1
150 of -1
160 of -1
170 of -1
180 of -1
190 of -1
200 of -1
210 of -1
220 of -1
230 of -1
240 of -1
250 of -1
260 of -1
270 of -1
280 of -1
290 of -1
300 of -1
310 of -1
320 of -1
0.000806989049606
HMM for 1.10.510.10.29.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.000806989049606
raw ISS is 1612
4098
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0
0.000806989049606
0.000806989049

HMM for 1.10.510.10.108.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.0010749868902
raw ISS is 1613
4100
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0
4930 of -1
0.00107565251744
HMM for 1.10.510.10.55.0 
HMM for 3.30.420.40.31.0 
norm ISS is 0.00107565251744
raw ISS is 1612
4099
http://xhits.cathdb.info/crosshits.php?sf2=1.10.510.10&sf1=3.30.420.40&db_source=Crosshits_v4_1_0


In [16]:
hmmids = list(hmms.values_list('id',flat = True))

In [46]:
key_v = sorted_D_norm[0]
print key_v
# D_temp = D_norm.toarray()

(2775, 19584, -3.2755416884013093)


In [129]:
# hmmid1,hmmid2, v = key_v
# hmmid1 += 1
# hmmid2 += 

hmm1 = HMMprofile.objects.get(id = hmmid1  )
hmm2 = HMMprofile.objects.get(id = hmmid2  )

inter_hitseq = hmm1.hits.all() & hmm2.hits.all()
hits1 = hmm1.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
hits2 = hmm2.hit4hmm2hsp_set.filter(target__in = inter_hitseq)


fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
print "intersection count is",inter_hitseq.count()
print  fmt %(hmm1,hmm1.cath_node.domain, p_hmmlen.findall(hmm1.text)[0], hmm1.hits.count() )
print fmt % (hmm2,hmm2.cath_node.domain, p_hmmlen.findall(hmm2.text)[0], hmm2.hits.count() )
for i,j in zip(hits1,hits2):
    try:
        node =  domain.objects.get(domain_id = i.target.acc).classification
    except:
        node = ''
    print i.target.acc,i.bitscore,
    print j.target.acc,j.bitscore
#     print node
    print i.start,j.start
    print i.end,j.end

intersection count is 0
HMM for 1.20.120.390.1.0 ,	 domain:2icwG01	 length:124,	 hitnum:2
HMM for 2.60.40.10.267.0 ,	 domain:2xy1A02	 length:94,	 hitnum:3480


In [86]:
hits1_ids = hmm1.hits.values_list('id',flat = 1)
hits2_ids = hmm2.hits.values_list('id',flat = 1)
inter_hitseq = set( hits1_ids ) & set( hits2_ids )

hits1 = list(hmm1.hit4hmm2hsp_set.filter(target__id__in = inter_hitseq))
hits2 = list(hmm2.hit4hmm2hsp_set.filter(target__id__in = inter_hitseq))

fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
for i,j in zip(hits1,hits2):
    print i.target.acc,i.bitscore,
    print j.target.acc,j.bitscore
#     print node
    print i.start,j.start
    print i.end,j.end

1gn0A00 73.6 1gn0A00 12.4
0 7
105 96


In [145]:
def compare_hits( hmm1,hmm2):
    hits1_ids = hmm1.hits.values_list('id')
    hits2_ids = hmm2.hits.values_list('id')
    inter_hitseq = set( hits1_ids ) & set( hits2_ids )
    hits1 = hmm1.hit4hmm2hsp_set.prefetch_related('target')
    hits2 = hmm2.hit4hmm2hsp_set.prefetch_related('target')
#     hits2 = hmm2.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
    fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
#     print "intersection count is",inter_hitseq.count()
    print "intersection count is",inter_hitseq.__len__()
    print  fmt %(hmm1,hmm1.cath_node.domain, p_hmmlen.findall(hmm1.text)[0], hmm1.hits.count() )
    print fmt % (hmm2,hmm2.cath_node.domain, p_hmmlen.findall(hmm2.text)[0], hmm2.hits.count() )
#     for i,j in zip(hits1,hits2):
    for hitid in inter_hitseq:
        i = hits1.get(target=hitid)
        j = hits2.get(target=hitid)
        try:
            node =  domain.objects.get(domain_id = i.target.acc).classification
        except Exception as e:
#             print e
            node = ''
        print i.target.acc,i.bitscore,
        print j.target.acc,j.bitscore
        if node: print node
        
        print i.start,j.start
        print i.end,j.end
        
        
compare_hits(hmm1,hmm2)
# hits1_ids = hmm1.hits.values_list('id')
# hits2_ids = hmm2.hits.values_list('id')

intersection count is 2594
HMM for 1.10.10.530.1.0 ,	 domain:2icwG02	 length:89,	 hitnum:2596
HMM for 2.60.40.10.230.0 ,	 domain:2xy2A02	 length:95,	 hitnum:3455
3juyA01 81.8 3juyA01 27.7
0 3
77 116
4lciL00 58.4 4lciL00 41.4
0 0
73 104
1zanL01 61.4 1zanL01 46.0
0 0
72 97
1cloL01 55.3 1cloL01 48.2
0 0
64 99
2z92A01 65.4 2z92A01 33.9
0 3
78 110
1rziA01 65.2 1rziA01 48.4
0 0
66 106
4nnpH01 85.0 4nnpH01 30.7
0 7
73 112
3cfiL00 77.5 3cfiL00 26.1
0 2
71 114
4ei6D01 40.9 4ei6D01 32.1
2 0
72 106
3p0yL01 60.8 3p0yL01 54.5
0 0
66 107
1etzB01 57.9 1etzB01 26.4
0 2
80 104
1rihL01 58.0 1rihL01 50.7
0 0
67 97
4eowH01 77.3 4eowH01 26.3
0 1
69 110
4whtA01 72.7 4whtA01 24.4
0 4
70 111
1eo8L01 50.6 1eo8L01 44.9
0 0
66 105
2xqyG01 90.3 2xqyG01 29.8
1 5
77 104
3mbxL01 64.1 3mbxL01 42.6
0 0
73 112
4jy6B01 69.0 4jy6B01 25.6
0 3
70 102
3iflH01 60.9 3iflH01 28.4
0 1
80 105
1pg7X01 71.1 1pg7X01 22.4
0 1
71 109
3i02A01 64.3 3i02A01 44.6
0 0
72 112
1z3gL01 55.1 1z3gL01 43.0
0 0
64 105
2x1qA00 71.6 2x1qA00 27.8
2

2adfL01 62.9 2adfL01 46.1
0 0
67 106
3t0wA00 60.1 3t0wA00 37.0
0 0
69 99
2z8vC00 18.4 2z8vC00 35.3
1 0
70 98
4ffvD01 72.4 4ffvD01 34.5
0 1
71 117
2nw2A01 37.4 2nw2A01 29.5
2.60.40.10.39.0
0 0
66 100
3zl4H01 77.8 3zl4H01 30.5
0 1
81 117
4hpoL01 56.5 4hpoL01 39.9
0 1
69 109
1ospH01 56.0 1ospH01 30.6
0 1
68 107
2j4wH01 78.3 2j4wH01 24.5
0 5
70 105
2imnA00 66.3 2imnA00 42.8
0 0
76 103
4p2qE01 31.2 4p2qE01 27.2
1 1
73 107
1sm3L01 39.3 1sm3L01 32.2
0 1
68 108
3kdmH01 83.2 3kdmH01 27.6
0 6
70 99
3rvvD01 69.4 3rvvD01 31.4
0 1
79 103
4nikB02 69.5 4nikB02 41.5
0 0
72 100
3so3B01 72.1 3so3B01 40.3
0 0
68 99
4jb9L01 54.2 4jb9L01 36.3
0 0
71 94
4k3dI01 71.5 4k3dI01 37.7
2.60.40.10.836.0
0 2
71 116
3u9pH01 75.8 3u9pH01 27.5
0 1
71 106
4a6yB01 90.4 4a6yB01 31.3
0 1
71 110
1nfdF01 71.2 1nfdF01 21.1
0 6
71 104
4webL01 65.2 4webL01 38.8
0 0
78 104
3v0aC00 77.3 3v0aC00 29.0
0 5
81 114
4f37K01 52.6 4f37K01 34.3
0 0
72 100
3cleH01 83.7 3cleH01 27.9
0 1
71 102
1nldH01 78.9 1nldH01 22.8
0 1
77 112
4m7lH01 75

62 86
4eupH01 39.5 4eupH01 15.1
1 0
76 102
4kmtH01 83.3 4kmtH01 21.7
0 0
69 103
2aj3A01 44.2 2aj3A01 43.5
0 0
66 101
2xwtA01 84.3 2xwtA01 23.6
0 0
69 104
3bjlB01 61.7 3bjlB01 43.9
0 0
70 109
4l3eD01 48.0 4l3eD01 39.2
0 0
70 108
3qpqF01 85.7 3qpqF01 34.7
0 1
70 100
3ncyP01 86.0 3ncyP01 22.3
0 0
71 103
3qcuH01 73.0 3qcuH01 20.1
0 0
69 106
1kenL01 64.3 1kenL01 43.3
0 0
69 98
1f3dL01 50.5 1f3dL01 33.2
0 0
72 101
2ipuK01 54.3 2ipuK01 34.1
0 0
72 100
4lp5A03 19.5 4lp5A03 57.8
2.60.40.10.620.0
1 1
41 87
4r26L01 38.8 4r26L01 32.0
2 1
61 106
1mj8L01 58.1 1mj8L01 38.4
0 0
71 100
3u4eA01 69.2 3u4eA01 23.6
0 4
66 105
4ebqL01 64.0 4ebqL01 43.1
0 0
65 100
4n1cB00 63.1 4n1cB00 52.9
0 0
67 106
4grwH00 57.7 4grwH00 25.5
0 5
70 110
3mugD01 70.1 3mugD01 17.9
0 4
90 103
4hhaA01 65.2 4hhaA01 38.8
0 0
72 94
4lsqL01 53.7 4lsqL01 48.8
0 0
70 102
1zglM01 45.7 1zglM01 27.7
0 1
72 100
1fh5H01 68.9 1fh5H01 22.4
0 4
64 100
1qokA01 75.8 1qokA01 32.6
0 0
72 106
3vi3E01 49.3 3vi3E01 34.7
0 0
71 100
4ki5E01 83.9 4ki5E

1qfwM00 57.0 1qfwM00 50.5
0 0
85 104
3uc0L01 62.1 3uc0L01 57.5
0 0
68 105
4pjxE01 38.0 4pjxE01 28.8
0 2
69 102
4i77L01 67.6 4i77L01 48.2
0 0
71 110
2dquL01 50.5 2dquL01 32.2
0 0
72 100
3o2vL01 57.7 3o2vL01 33.8
0 0
72 100
4gmsH01 76.2 4gmsH01 26.0
0 1
74 104
1xgtB01 61.4 1xgtB01 33.3
0 1
78 112
2jixD01 84.0 2jixD01 35.7
0 1
79 101
4g5zL01 61.4 4g5zL01 46.0
0 0
73 97
4lhjB00 63.7 4lhjB00 34.4
0 5
70 110
1tzhA01 65.5 1tzhA01 56.1
0 0
66 106
1bbjH01 82.7 1bbjH01 31.1
0 0
71 113
1faiH01 81.3 1faiH01 29.5
0 0
81 111
2qadH01 70.9 2qadH01 21.3
0 0
69 107
4dn3H01 71.8 4dn3H01 25.4
0 0
69 103
3okdA01 68.3 3okdA01 43.5
0 0
73 112
4dqoH01 70.1 4dqoH01 17.9
0 4
90 103
4lveB00 68.7 4lveB00 41.2
0 0
73 112
1za3A01 64.2 1za3A01 56.6
0 0
72 106
3eo9L01 66.5 3eo9L01 56.9
0 0
71 106
4hf5H01 78.8 4hf5H01 28.7
0 4
68 112
3gizL01 69.0 3gizL01 43.0
0 0
72 103
3ra7L01 60.6 3ra7L01 46.4
0 0
71 95
3gkzA02 67.1 3gkzA02 47.4
0 1
69 99
1nakL01 59.1 1nakL01 33.2
0 0
72 104
1afvL01 63.4 1afvL01 34.6
0 0
75 109
3q6g

1mfaL01 37.7 1mfaL01 33.7
0 0
67 108
1jvkA01 56.6 1jvkA01 38.7
0 0
71 110
3l95B01 85.8 3l95B01 29.9
0 5
80 104
2p44B00 55.1 2p44B00 23.9
0 5
70 121
3fo0L01 53.6 3fo0L01 33.6
0 0
70 97
3k7uA00 62.4 3k7uA00 31.7
0 2
69 112
1jhlH00 89.7 1jhlH00 30.5
0 0
71 116
4u1gC01 75.5 4u1gC01 42.5
0 0
74 101
2orbM01 64.0 2orbM01 31.6
0 0
70 106
1rzfL01 63.4 1rzfL01 39.4
0 0
69 101
1ndgB01 59.6 1ndgB01 33.2
0 1
71 110
3csyB01 63.8 3csyB01 40.5
0 0
73 104
3mlvL01 54.0 3mlvL01 44.1
0 1
62 107
4qwwC01 58.8 4qwwC01 47.1
0 0
65 104
1tziB01 72.3 1tziB01 30.0
0 5
78 109
2w9dL01 55.0 2w9dL01 44.3
0 0
71 96
3g6jE01 64.0 3g6jE01 53.8
0 0
72 107
3cfdB01 66.9 3cfdB01 32.0
0 1
79 109
3u2sH01 69.2 3u2sH01 23.6
0 4
66 105
4otxH01 88.3 4otxH01 27.1
0 1
71 112
2qhlD00 50.1 2qhlD00 30.8
2.60.40.10.65.0
2 2
69 107
1k6qL01 57.4 1k6qL01 49.6
0 0
66 98
4gq9H01 84.8 4gq9H01 31.8
0 0
71 117
3ijhB01 76.9 3ijhB01 28.1
0 5
72 104
3cmoX01 57.0 3cmoX01 46.7
0 0
74 106
3vg9C01 78.3 3vg9C01 29.9
0 0
71 108
3rpiB01 55.2 3rpiB01 49.1

...

2y07L01 39.3 2y07L01 29.7
0 0
67 107
4pjeF01 54.6 4pjeF01 26.1
0 1
77 107
3qeuB01 37.7 3qeuB01 23.1
2 1
71 112
3o2dL01 67.4 3o2dL01 44.7
0 0
75 112
2bnuB01 46.4 2bnuB01 23.3
0 0
68 99
2h1pH01 70.5 2h1pH01 25.9
0 7
70 108
1jpsH01 81.5 1jpsH01 28.9
0 5
70 115
2x7lA01 73.3 2x7lA01 29.6
0 3
72 104
3tt3L01 66.5 3tt3L01 41.4
0 0
70 110
1onqA02 5.7 1onqA02 11.1
6 16
50 67
4uv4L01 57.9 4uv4L01 33.8
0 0
74 98
4kaqL01 54.0 4kaqL01 45.4
0 0
66 99
4rqqH01 65.1 4rqqH01 20.8
0 1
71 106
4gftB00 56.7 4gftB00 29.2
0 2
71 114
1gafL01 47.2 1gafL01 41.3
0 0
69 104
4ms8C01 62.0 4ms8C01 39.1
2.60.40.10.228.0
0 3
76 116
4jlrB01 65.0 4jlrB01 46.4
0 0
70 94
1a5fH01 86.0 1a5fH01 30.2
0 1
70 100
4hemE00 65.7 4hemE00 29.9
0 5
70 110
1dzbA02 61.5 1dzbA02 43.3
0 0
66 95
3v0wH01 79.1 3v0wH01 30.1
0 6
72 118
4dkaA00 67.4 4dkaA00 31.8
0 2
70 109
2xqbH01 82.2 2xqbH01 22.8
0 0
71 107
4jffD01 42.8 4jffD01 37.0
0 0
73 108
3aazA01 67.9 3aazA01 33.1
0 0
73 111
2fb4H01 80.0 2fb4H01 31.0
0 3
70 110
2a9nH00 73.5 2a9nH00 26.2
0

3arbD01 52.7 3arbD01 31.5
0 0
75 105
3tyfC01 30.6 3tyfC01 52.7
0 1
74 104
1xfpA00 61.9 1xfpA00 27.1
0 3
70 109
4nrxB01 69.0 4nrxB01 56.4
0 0
66 97
3kr3H01 76.6 3kr3H01 28.2
0 9
70 109
3ab0C00 57.0 3ab0C00 44.2
0 0
66 97
3cxdH01 78.9 3cxdH01 22.2
0 6
72 115
3c5sB01 70.0 3c5sB01 27.9
0 3
72 114
1l7tH01 66.5 1l7tH01 24.6
0 6
69 100
2p4aB00 55.9 2p4aB00 33.3
0 5
70 120
4jpwL01 51.7 4jpwL01 46.3
0 0
69 101
1a7pL00 65.9 1a7pL00 51.7
0 0
66 98
1cd8A00 15.9 1cd8A00 32.2
1 0
75 108
2qqkL01 63.9 2qqkL01 52.7
0 0
72 104
2f53D01 35.3 2f53D01 42.5
0 0
71 104
4g6mL01 64.6 4g6mL01 48.0
0 0
66 93
1ggiL01 68.3 1ggiL01 42.4
0 0
75 109
1fbiP01 52.9 1fbiP01 44.2
0 0
69 94
3m8oH01 71.6 3m8oH01 27.2
0 5
69 118
3qumH01 91.3 3qumH01 23.3
0 0
69 101
2g75A01 75.1 2g75A01 29.6
0 0
69 117
2b1hH01 77.6 2b1hH01 22.6
0 0
71 106
2vl5B01 67.8 2vl5B01 43.9
0 0
81 107
2zpkM01 37.1 2zpkM01 34.2
0 0
66 108
3ntcH01 82.0 3ntcH01 20.4
2.60.40.10.111.0
0 0
80 99
3b9kC01 59.7 3b9kC01 45.3
0 0
70 106
4lgsB00 63.7 4lgsB00 30.3
0

2j6eH01 73.6 2j6eH01 28.6
0 11
92 112
1j1xL00 60.9 1j1xL00 50.4
0 0
66 97
4f33D01 75.1 4f33D01 21.7
0 1
71 105
4bz2H01 82.9 4bz2H01 28.8
0 1
70 104
4ocxL01 51.8 4ocxL01 31.8
0 0
72 100
1d5iL01 57.9 1d5iL01 47.1
0 0
65 105
3hc3H01 91.1 3hc3H01 27.6
0 0
69 114
1mcbA01 61.2 1mcbA01 37.4
1 0
71 110
2r0lH01 78.3 2r0lH01 27.1
0 5
71 99
2ck0L01 69.5 2ck0L01 46.2
0 0
77 112
12e8H01 75.1 12e8H01 36.5
0 0
71 108
3dusA00 68.9 3dusA00 42.7
0 0
73 111
1ap2B00 81.8 1ap2B00 28.9
0 0
69 105
4okvA01 88.4 4okvA01 29.8
0 0
71 112
3fmgH01 76.5 3fmgH01 24.4
0 0
81 105
3mckB01 62.6 3mckB01 30.8
0 2
81 106
2nr6D01 79.3 2nr6D01 31.1
0 1
69 115
2r56L01 63.3 2r56L01 51.3
0 0
72 107
4f9lC02 81.7 4f9lC02 32.8
0 0
66 104
1f97A01 14.1 1f97A01 60.2
2.60.40.10.401.0
2 0
60 102
4llyA01 74.8 4llyA01 28.8
0 5
69 100
4eneD01 52.6 4eneD01 45.5
0 0
64 101
3qrgH01 67.2 3qrgH01 31.3
0 2
79 100
1mlbB01 79.9 1mlbB01 28.4
0 0
72 115
3na9H01 79.3 3na9H01 20.0
0 0
69 107
1fj1A01 63.5 1fj1A01 47.0
0 0
69 106
3i9gH01 68.4 3i9gH01 2

70 106
4p46A01 46.1 4p46A01 44.3
0 0
68 107
4laqH00 40.7 4laqH00 32.2
0 0
53 101
2nr6C01 57.3 2nr6C01 39.8
0 0
72 106
1deeB01 83.2 1deeB01 27.5
0 4
70 102
3sobL01 63.8 3sobL01 53.6
0 0
66 106
2uziH00 77.6 2uziH00 27.6
0 9
70 100
2j88L01 58.4 2j88L01 58.4
0 0
72 106
4g8fB01 47.1 4g8fB01 17.5
0 0
74 104
3mffB01 44.2 3mffB01 19.6
0 0
73 102
1yeeH01 73.9 1yeeH01 24.2
0 1
71 100
4u6vL01 66.5 4u6vL01 51.0
0 0
71 106
4aeiH01 76.7 4aeiH01 23.6
0 6
70 105
3ujjL01 60.3 3ujjL01 49.1
0 2
67 107
4kucH01 84.6 4kucH01 27.3
0 0
66 103
3qjfA01 26.8 3qjfA01 41.8
1 2
73 104
4py7B01 65.0 4py7B01 53.4
0 0
69 99
1f4wL01 40.4 1f4wL01 31.5
0 0
66 108
1c12B01 70.1 1c12B01 33.5
0 1
79 119
4pjgF01 54.6 4pjgF01 26.7
0 1
77 104
1jv5A00 53.8 1jv5A00 46.9
0 0
71 102
3gnmL01 53.9 3gnmL01 33.0
0 0
72 106
2dqhH00 64.0 2dqhH00 31.8
0 1
79 111
3lhpI00 71.9 3lhpI00 30.1
0 1
71 107
4fqjH01 74.1 4fqjH01 23.3
0 0
70 105
1nccH01 88.2 1nccH01 26.2
0 2
70 118
2cdfB01 41.7 2cdfB01 21.5
0 0
70 104
1aj7L01 50.2 1aj7L01 44.2
0 0
72

4g8fA01 37.4 4g8fA01 32.2
1 1
67 102
4pjfE01 38.0 4pjfE01 29.0
0 2
69 102
4nzrH01 52.6 4nzrH01 24.8
0 2
77 110
1xiwC00 55.3 1xiwC00 45.7
0 0
73 107
1mjjH01 90.3 1mjjH01 30.1
0 1
71 100
3wd5H01 77.7 3wd5H01 25.5
0 9
70 109
1pskH01 83.8 1pskH01 30.9
0 1
71 113
1i7zB01 88.2 1i7zB01 26.7
0 0
70 117
4c57D00 73.6 4c57D00 27.5
0 2
70 121
3utzD01 50.8 3utzD01 34.2
0 0
72 111
3phoB01 76.6 3phoB01 28.1
0 5
72 108
2ap2A00 67.8 2ap2A00 50.1
1 2
89 104
1qfuL01 59.1 1qfuL01 34.9
0 0
72 102
1pewA00 40.1 1pewA00 38.8
0 1
65 106
3q6fD01 78.8 3q6fD01 27.7
0 2
70 106
3mv8E01 59.2 3mv8E01 33.1
0 0
72 103
3quyC01 32.7 3quyC01 48.6
0 0
74 101
1mexL01 51.2 1mexL01 44.3
0 0
81 105
3nfjH01 43.7 3nfjH01 33.4
0 0
74 106
2ih3A01 81.3 2ih3A01 31.7
0 0
71 117
4bh8B01 84.9 4bh8B01 29.7
0 0
70 107
1s3kH01 83.0 1s3kH01 30.4
0 4
70 108
1cdyA01 13.3 1cdyA01 42.9
0 0
35 97
4gmsJ01 76.2 4gmsJ01 26.0
0 1
74 104
2xtjB00 60.9 2xtjB00 54.2
0 0
67 106
3u7aA00 64.2 3u7aA00 49.4
0 1
74 106
4i13B00 52.2 4i13B00 27.6
0 3
70 111
2o